##📘 RPA 개요 및 실습 예제

###🧩 1. RPA 개요 (Robotic Process Automation)

| 항목   | 설명                                                              |
| ---- | --------------------------------------------------------------- |
| 정의   | 사람이 반복적으로 수행하는 업무를 소프트웨어 봇이 자동으로 처리하는 기술                        |
| 목적   | 반복 업무 자동화 → 효율성 증대, 실수 감소, 비용 절감                                |
| 특징   | 코드 작성 없이 GUI 기반으로 구성 가능, 기존 시스템 변경 불필요                          |
| 대표 툴 | UiPath, Automation Anywhere, Microsoft Power Automate, Robocorp |

###💡 2. RPA 적용 사례

| 업무     | 적용 예시                     |
| ------ | ------------------------- |
| 회계/재무  | 세금 계산서 수집 및 입력, 전표 처리 자동화 |
| 고객 서비스 | 이메일 문의 자동 응답, FAQ 응답 자동화  |
| HR     | 지원자 이력서 분류, 근태 데이터 수집     |
| IT 운영  | 시스템 로그 백업, 정기 리포트 발송      |

###🧪 3. 실습 예제 – Python + RPA (RPA-Python 연동 실습)
예제 주제: 엑셀에서 고객 이메일 주소 읽고, 자동으로 이메일 발송하기

📂 준비물

- pandas, smtplib, openpyxl

- 이메일 발송 계정 (Gmail 기준)

- 고객 정보가 담긴 Excel (customers.xlsx)

📄 customers.xlsx 예시

| 이름  | 이메일                                           |
| --- | --------------------------------------------- |
| 김지훈 | [jihun@example.com](mailto:jihun@example.com) |
| 박민지 | [minji@example.com](mailto:minji@example.com) |


💻 코드 예제

✅ 2. 앱 비밀번호 생성

https://myaccount.google.com/security

앱 비밀번호 설정 페이지 접속
(2단계 인증이 활성화되어 있어야 접속 가능)

1. 앱 선택 → "메일"

2. 생성 버튼 클릭

3. 표시된 16자리 비밀번호 복사 → 코드에 사용

In [8]:
import pandas as pd
# 엑셀에서 데이터 읽기
df = pd.read_excel("customers.xlsx")
df

,이름,이메일
0,윤소율,soyul9175@naver.com


In [2]:
import pandas as pd
import smtplib
from email.message import EmailMessage
from google.colab import userdata

# 엑셀에서 데이터 읽기
df = pd.read_excel("customers.xlsx")

# 이메일 서버 설정 (Gmail)
EMAIL_ADDRESS = "soyul9175@gmail.com"
EMAIL_PASSWORD = userdata.get("MAIL_KEY")


# SMTP 서버 연결
with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

    for idx, row in df.iterrows():
        msg = EmailMessage()
        msg["Subject"] = "감사합니다, 고객님!"
        msg["From"] = EMAIL_ADDRESS
        msg["To"] = row["이메일"]

        msg.set_content(f"{row['이름']}님, 저희 서비스를 이용해주셔서 감사합니다.")

        smtp.send_message(msg)
        print(f"이메일 발송 완료: {row['이름']} ({row['이메일']})")


이메일 발송 완료: 윤소율 (soyul9175@naver.com)


### 🧠 LLaMA 기반 맞춤 이메일 생성 실습

📌 목적

고객 이름과 상황에 따라 개인화된 이메일 메시지를 LLM이 자동 생성 → SMTP로 발송

🧰 실습 구성

- 입력: 고객 이름, 상황 정보 (예: 가입, 구매, 문의 등)

- 출력: 맞춤형 이메일 메시지

- 도구: transformers 라이브러리, LLaMA 모델 (또는 distilGPT2 등 경량 모델로 대체 가능), SMTP 메일 발송

🔧 필요 라이브러리 설치

In [11]:
!pip install transformers accelerate pandas

📄 예제 코드 (Hugging Face 모델 기반)

huggingface: https://huggingface.co/ 접속

User permissions (So-Yul) All check -> token 발행

meta-llama/Llama-3.1-8B-Instruct 검색

아코디언 찾아서 펼쳐서 개인정보/기관 인증 후 사용 가능


In [26]:
from huggingface_hub import login

# 본인의 Hugging Face 토큰 입력
token = userdata.get("LLM_TOKEN")
login(token)

1. LLaMA3 기반 텍스트 생성 함수

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# 모델 불러오기 (최초 실행 시 시간 소요)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


def extract_email_body(full_text, prompt):
    # 프롬프트 길이만큼 자르고, 앞뒤 공백 제거
    return full_text[len(prompt):].strip()

def generate_email(name, context):
    # 프롬프트 정의
    prompt = f"""
    아래 조건에 맞춰 고객에게 보낼 감사 이메일을 한글로 작성해줘:

    고객 이름: {name}
    상황: {context}
    이메일 내용:
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=200, temperature=0.7)
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)

    # 메시지 부분만 추출
    return extract_email_body(full_text, prompt)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

 2. 고객 목록 및 이메일 내용 생성

In [28]:
import pandas as pd

# 고객 정보
df = pd.DataFrame({
    "이름": ["소율"],
    "이메일": ["soyul9175@gmail.com"],
    "상황": ["서비스 가입 완료"]
})

# 각 고객에게 이메일 내용 생성
df["본문"] = df.apply(lambda row: generate_email(row["이름"], row["상황"]), axis=1)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


📧 3. 이메일 발송 함수

In [29]:
import smtplib
from email.message import EmailMessage

# 이메일 서버 설정 (Gmail)
EMAIL_ADDRESS = "soyul9175@gmail.com"
EMAIL_PASSWORD = userdata.get("MAIL_KEY")

def send_email(to, subject, body):
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = EMAIL_ADDRESS
    msg["To"] = to
    msg.set_content(body)

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        smtp.send_message(msg)

🚀 4. 전체 자동화 실행

In [30]:
for _, row in df.iterrows():
    subject = f"{row['이름']}님 감사합니다!"
    body = row["본문"]
    send_email(row["이메일"], subject, body)
    print(f"✅ {row['이름']}님에게 이메일 발송 완료")

✅ 소율님에게 이메일 발송 완료


##✅ 전체 통합 코드: LLaMA3 기반 요약 + 한글 리포트 PDF + 이메일 발송

In [ ]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.7 MB/s eta 0:00:00


###한글 폰트 설치

In [ ]:
!apt-get -y update >/dev/null
!apt-get -y install fonts-nanum >/dev/null

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager as fm
import numpy as np

# 설치된 폰트 경로(예: NanumGothic)
font_path_candidates = [
    "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",     # fonts-nanum
    "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc",  # fonts-noto-cjk
]

# 사용 가능한 폰트 경로 선택
font_path = next((p for p in font_path_candidates if fm.os.path.exists(p)), None)
if font_path:
    fm.fontManager.addfont(font_path)

# ✅ 폰트 패밀리 지정 (우선 Nanum, 없으면 Noto)
matplotlib.rcParams["font.family"] = "NanumGothic" if "NanumGothic" in [f.name for f in fm.fontManager.ttflist] else "Noto Sans CJK KR"

# ✅ 마이너스 기호 깨짐 방지
matplotlib.rcParams["axes.unicode_minus"] = False

### LLAMA3 요약 함수

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def generate_llama3_summary(input_text):
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

    # 모델 & 토크나이저 불러오기
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # 프롬프트 구성
    prompt = f"""
    다음 내용을 요약하여 비즈니스 분석 보고서 초안을 작성해줘.

    "{input_text}"

    보고서:
    """

    # 토큰화 및 생성
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        top_p=0.9
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 보고서 부분만 추출
    if "보고서:" in result:
        return result.split("보고서:")[-1].strip()
    return result.strip()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.utils import ImageReader
from datetime import datetime
import smtplib
from email.message import EmailMessage

# 폰트 등록 (한글 지원)
pdfmetrics.registerFont(TTFont("Nanum", "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"))

# 샘플 데이터
data = {
    "날짜": ["2025-05-01", "2025-05-01", "2025-05-02", "2025-05-02", "2025-05-03"],
    "제품명": ["A", "B", "A", "C", "B"],
    "수량": [10, 5, 3, 7, 8],
    "금액": [10000, 15000, 3000, 7000, 12000]
}
df = pd.DataFrame(data)
df["날짜"] = pd.to_datetime(df["날짜"])

# 요약 통계
summary = df.groupby("날짜").agg({"수량": "sum", "금액": "sum"}).reset_index()

# 시각화
plt.rcParams['font.family'] = 'NanumGothic'
plt.figure(figsize=(6, 4))
plt.plot(summary["날짜"], summary["금액"], marker="o")
plt.title("일자별 매출 추이")
plt.xlabel("날짜")
plt.ylabel("매출 (원)")
plt.grid(True)
plt.tight_layout()
plot_path = "sales_plot_final.png"
plt.savefig(plot_path)
plt.close()

In [ ]:
# 요약 텍스트
total_sales = df["금액"].sum()
total_qty = df["수량"].sum()
top_day = summary.loc[summary["금액"].idxmax(), "날짜"].strftime("%Y-%m-%d")
summary_input = f"총 매출은 {total_sales:,}원이었고, 판매 수량은 {total_qty}개입니다. 가장 높은 매출은 {top_day}에 발생했습니다."
##llama_summary = f"{summary_input} 해당 결과를 바탕으로 향후 재고 확보 전략을 보완하는 것이 권장됩니다."
llama_summary = generate_llama3_summary(summary_input)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1025: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
llama_summary

'제목: 2022년 5월 1일 매출 분석 보고서\n     요약: 2022년 5월 1일 총 매출은 47,000원으로, 33개의 판매 수량으로 발생했습니다. 가장 높은 매출은 2025-05-01에 발생했습니다.\n     분석: \n       - 총 매출은 47,000원으로, 판매 수량은 33개입니다.\n       - 가장 높은 매출은 2025-05-01에 발생했습니다.\n     추천: \n       - 판매 수량을 증가시키는 방안을 찾을 필요가 있습니다.\n       - 가장 높은 매출을 발생시킨 날짜를 분석하여 해당 제품의 판매 전략을 개선할 필요가 있습니다.\n\n    참고: \n    - 이 보고서는 2022년 5월 1일의 매출 분석을 목표로 작성되었습니다.\n    - 이 보고서'

In [ ]:
# 📄 PDF 생성
pdf_path = "sales_report_final.pdf"
c = canvas.Canvas(pdf_path, pagesize=A4)
c.setFont("Nanum", 14)
width, height = A4

c.drawString(50, height - 50, "📊 판매 리포트")
c.setFont("Nanum", 12)
c.drawString(50, height - 90, "[요약 정보]")
text = c.beginText(50, height - 110)
text.setFont("Nanum", 11)
text.textLines(summary_input)
c.drawText(text)

c.drawString(50, height - 180, "[AI 요약]")
text2 = c.beginText(50, height - 200)
text2.setFont("Nanum", 11)
text2.textLines(llama_summary)
c.drawText(text2)

# 이미지 추가
c.drawImage(ImageReader(plot_path), 50, height - 700, width=500, preserveAspectRatio=True)
c.save()

In [ ]:
# ✔️ 자동 이메일 전송 (필요 시 주석 해제 후 사용)
def send_email_with_report(to_email, from_email, app_password, subject, body, attachment_path):
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = from_email
    msg["To"] = to_email
    msg.set_content(body)

    with open(attachment_path, "rb") as f:
        msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename="sales_report.pdf")

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login(from_email, app_password)
        smtp.send_message(msg)

# 사용 예시
send_email_with_report("jikim@imguru.co.kr", "jikim@imguru.co.kr", EMAIL_PASSWORD, "주간 리포트", "보고서를 확인하세요.", pdf_path)

pdf_path


'sales_report_final.pdf'

##🚗 자동차 전장 분야에서의 RPA 활용 시나리오

| 활용 분야           | 자동화 내용                           |
| --------------- | -------------------------------- |
| 🔍 부품 품질 검사 리포트 | 검사 로그 수집 → 요약 보고서 생성 → 자동 이메일 발송 |
| 📦 BOM 관리       | 설계 변경에 따른 BOM 자동 업데이트 및 승인 메일 발송 |
| 🛠 고객 CS 대응     | 정비 요청 자동 분류 및 CRM 시스템 등록         |
| 🔧 부품 재고 자동화    | ERP 재고 변화 감지 → 재주문 메일 자동 전송      |
| 📑 인증 문서 자동화    | KS 인증·EMC 테스트 리포트 작성 자동화         |


###📄 예시 시나리오: "부품 품질 검사 보고서 자동화"

목표: 전장부품 검사 로그 파일 → 요약 → PDF 생성 → 이메일 전송


📦 1. 입력 파일 (예시: inspection_log.xlsx)

| 검사일        | 부품코드    | 검사결과 | 불량사유  |
| ---------- | ------- | ---- | ----- |
| 2025-05-01 | VCU-001 | 불량   | 납땜불량  |
| 2025-05-01 | VCU-001 | 양호   | -     |
| 2025-05-01 | VCU-002 | 불량   | 전압불안정 |


🧠 2. Python 기반 RPA 자동화 예제

In [31]:
import pandas as pd
from fpdf import FPDF
from email.message import EmailMessage
import smtplib

# 1. 검사 로그 로드
df = pd.read_excel("inspection_log.xlsx")

# 2. 불량 통계 요약
summary = df[df["검사결과"] == "불량"].groupby("부품코드").size().reset_index(name="불량건수")

# 3. 요약 텍스트 생성
text = "[불량 부품 요약]\n"
for _, row in summary.iterrows():
    text += f"- {row['부품코드']}: {row['불량건수']}건\n"

# 4. PDF 리포트 생성
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, "전장부품 검사 결과 보고서", ln=True, align="C")
pdf.ln(10)
pdf.multi_cell(0, 10, text)
report_path = "inspection_report.pdf"
pdf.output(report_path)

# 5. 이메일 자동 전송 (사내 품질팀)
def send_report_email():
    msg = EmailMessage()
    msg["Subject"] = "📋 품질검사 리포트 자동 전송"
    msg["From"] = "your_email@example.com"
    msg["To"] = "quality_team@example.com"
    msg.set_content("첨부된 자동 생성 리포트를 확인해 주세요.")

    with open(report_path, "rb") as f:
        msg.add_attachment(f.read(), maintype="application", subtype="pdf", filename="inspection_report.pdf")

    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        smtp.login("your_email@example.com", "your_app_password")
        smtp.send_message(msg)

# send_report_email()  # 실제 실행 시 사용


ModuleNotFoundError: No module named 'fpdf'